In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# 定义 CNN 结构
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(2, 2)  # 每次池化减半
        self.dropout = nn.Dropout(0.25)

        # **自动计算全连接层的输入尺寸**
        self.fc1_input_size = self._get_conv_output((1, 28, 28))  # 计算输入尺寸
        self.fc1 = nn.Linear(self.fc1_input_size, 128)
        self.fc2 = nn.Linear(128, 10)

    def _get_conv_output(self, shape):
        """ 计算卷积层输出的特征图大小，以便正确设置全连接层 """
        x = torch.rand(1, *shape)  # 创建一个假的输入
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.maxpool(x)  # 经过池化
        return x.numel()  # 计算展平后的元素个数

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.maxpool(x)  # 经过池化后尺寸缩小

        # **自动计算展平后的形状**
        x = torch.flatten(x, 1)
        # print(f"Flattened shape: {x.shape}")  # 方便调试

        x = self.dropout(x)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 训练参数
batch_size = 64
learning_rate = 0.001
epochs = 5

# 数据预处理
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

# 下载 MNIST 数据集
train_dataset = torchvision.datasets.MNIST(root="./data", train=True, transform=transform, download=True)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# 初始化模型、损失函数和优化器
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print("开始训练模型...")
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss / len(train_loader):.4f}")

# 保存模型权重
torch.save(model.state_dict(), "mnist_cnn.pth")
print("训练完成，模型已保存为 mnist_cnn.pth")


开始训练模型...


KeyboardInterrupt: 